# Spotify Recommendation - Finding your music taste!

## Import necessary libraries

In [1]:
import pandas as pd
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import requests
import time
from pandas import json_normalize

## Using Credentials to access OAuth for spotify

In [2]:
# Your credentials
client_id = '0af53b02c90245839d0f6d6f3bf45fb1'
client_secret = '89783cd95eac43eebbfc7f2e2a885dbb'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://accounts.spotify.com/api/token',
    client_id=client_id,
    client_secret=client_secret,
    include_client_id=True
)

## Ideas for model

Before we start, we need to consider a couple of things when recommending songs:

- Is the track available for specified country?

- Did the user dislike similar tracks or this specified track?

Features for the model:

- Maybe consider getting input from users of language/country of song you want?

## Initializing Token and Getting API Request

In [3]:
# The headers to provide the access token for authentication
headers = {
    'Authorization': f'Bearer {token["access_token"]}',     # OAuth 2.0 Bearer token for authorization
    'Content-Type': 'application/json'                      # Indicates the media type of the resource
}

In [4]:
token_expiry = token['expires_at'] # gives time of expiry in total seconds
#token_expiry_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(token_expiry)) # converting to readable time format

we need to check if time.time() is going to exceed the token_expiry_date. If so then renew the token!

In [5]:
# void function to get new token if needed
def refresh_token_if_needed():
    global token, token_expiry  # Declare token and token_expiry as global variables
    # Check if the current time is within 60 seconds of expiry or past it
    if time.time() > token_expiry - 60:
        client = BackendApplicationClient(client_id=client_id)
        oauth = OAuth2Session(client=client)
        
        # Fetch a new token and update the global token variable
        token = oauth.fetch_token(
            token_url='https://accounts.spotify.com/api/token',
            client_id=client_id,
            client_secret=client_secret,
            include_client_id=True
        )
        
        # Calculate and update the global token_expiry variable
        token_expiry = time.time() + token['expires_in']


## Finding songs based on user chosen track

In [6]:
song_title = "Self Care"
artist_name = "Mac Miller"
# check if search is case sensitive?

function that search user defined song.

The search is not case sensitive.

Ex: artist_name = mac miller, song_title='whats the use'

It will still give the song. Original name is "Mac Miller", "What's the use?"

In [17]:
def search_song(artist_name="", song_title=""):
    ###
    ### given artist_name and song_tile the functino search_song will output corresponding song based on input
    ###
    url = "https://api.spotify.com/v1/search"
    refresh_token_if_needed()
    headers = {
        'Authorization': f'Bearer {token["access_token"]}',  # specify token
        'Content-Type': 'application/json'  # specify data type that you receive
    }
    # dictionary format for search parameter
    # https://developer.spotify.com/documentation/web-api/reference/search 
    # use the link above for a documentation guide for search parameters
    search_query = f'track:{song_title} artist:{artist_name}'.strip()
    search_params = {
        'q': search_query if search_query else 'year:0000',  # A default query that returns minimal results
        'type': 'track',  # specifies that you are searching for tracks
        'limit': 1  # limits the response to the top 1 most relevant result
    }

    response = requests.get(url, headers=headers, params=search_params).json()
    # You might want to add error handling here to manage potential API errors

    # Extracting the most relevant song; if no songs are found, returns None
    # return as pd data frame
    return json_normalize(response['tracks']['items'][0]) if response['tracks']['items'] else None

Searching song based on song title and artist name

In [21]:
df = search_song('mac miller', 'whats the use')
df

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,288640,True,https://api.spotify.com/v1/tracks/2dgrYdgguVZK...,2dgrYdgguVZKeCsrVb9XEs,False,What's the Use?,69,...,5wtE5aLX5r7jOosmPhJhhk,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Swimming,2018-08-03,day,13,album,spotify:album:5wtE5aLX5r7jOosmPhJhhk,USWB11801213,https://open.spotify.com/track/2dgrYdgguVZKeCs...


In [22]:
df['artist_name'] = df['artists'][0][0]['name']
result_df = df[['name', 'artist_name']]
result_df


,name,artist_name
0,What's the Use?,Mac Miller


## Recommending songs based on user specified playlist

user will enter playlist link and AI model will return list of songs.

In [10]:
playlist_url = "https://open.spotify.com/playlist/79a7eH7yE02kUXBcTczEvk"

In [11]:
# given a url for spotify plyalist, only take the playlist ID which is at the end of the url
playlist_id = playlist_url.split('/')[-1]
# refresh token function called, this will refresh token if token time is close to expiry time
refresh_token_if_needed()

tracks = []
url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'

while url:
    response = requests.get(url, headers={
        'Authorization': f'Bearer {token["access_token"]}',
        'Content-Type': 'application/json'
    }).json()
    
    # Error handling: Break the loop if the response contains an error message.
    if 'error' in response or not response['items']:
        print("Failed to fetch data:", response.get('error', {}))
        break
    
    tracks += [
        {
            'track_name': item['track']['name'],
            'artist_name': item['track']['artists'][0]['name'],
            'track_id': item['track']['id']
        }
        for item in response['items'] if item['track']
    ]
    
    # Spotify provides a 'next' URL to paginate through playlists
    url = response.get('next')

pd.DataFrame(tracks)

,track_name,artist_name,track_id
0,Come Over,DEAN,1ieDhJfetL1wUFTC0YIriP
1,I Love It,DEAN,2qEZldTbAPtI3LNs5IAten
2,D (Half Moon),DEAN,1iEzGFEux7t1Wk41LOaCCr
3,All I Wanna Do (feat. Hoody & 로꼬 Loco) [Korean...,Jay Park,6jLsYKGOBRBRvdowIzuyUf
4,Solo (feat. Hoody),Jay Park,4Y6Voc2xpc4lPD5iKpN4o2
...,...,...,...
894,DAY OFF,Sik-K,0dbwg8PpOML96BwB7VjM7W
895,Ripple,Sophiya,0yzzhnKn2lPNGa8wHcSGBh
896,ROSETTA (Feat. MILLI),pH-1,3Kf4pY9LwV1cptb4HDNFyk
897,LUST (iykyk) (Feat. DeVita),pH-1,4J3p2odZsCbjgsSmM3gqBJ


In [12]:
url =  f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
response = requests.get(url, headers={
        'Authorization': f'Bearer {token["access_token"]}',
        'Content-Type': 'application/json'
    }).json()

In [23]:
df = json_normalize(response['items'])
df

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,...,track.id,track.is_local,track.name,track.popularity,track.preview_url,track.track,track.track_number,track.type,track.uri,video_thumbnail.url
0,2017-03-30T13:55:40Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,single,[{'external_urls': {'spotify': 'https://open.s...,...,1ieDhJfetL1wUFTC0YIriP,False,Come Over,0,None,True,2,track,spotify:track:1ieDhJfetL1wUFTC0YIriP,None
1,2017-03-30T13:56:42Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,2qEZldTbAPtI3LNs5IAten,False,I Love It,0,None,True,6,track,spotify:track:2qEZldTbAPtI3LNs5IAten,None
2,2017-03-30T13:56:53Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,1iEzGFEux7t1Wk41LOaCCr,False,D (Half Moon),0,None,True,5,track,spotify:track:1iEzGFEux7t1Wk41LOaCCr,None
3,2017-03-30T13:58:36Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,6jLsYKGOBRBRvdowIzuyUf,False,All I Wanna Do (feat. Hoody & 로꼬 Loco) [Korean...,0,None,True,4,track,spotify:track:6jLsYKGOBRBRvdowIzuyUf,None
4,2017-03-30T13:59:13Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,4Y6Voc2xpc4lPD5iKpN4o2,False,Solo (feat. Hoody),0,None,True,6,track,spotify:track:4Y6Voc2xpc4lPD5iKpN4o2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2017-05-26T10:17:31Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,6WX68Y7z07oQlz3VolcGXP,False,DA DA DA (Feat. Hoody),0,None,True,8,track,spotify:track:6WX68Y7z07oQlz3VolcGXP,None
96,2017-05-26T10:17:42Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,5K209mgnrnMPLNIluT9m77,False,Still (Feat. Crush),0,None,True,9,track,spotify:track:5K209mgnrnMPLNIluT9m77,None
97,2017-05-26T10:17:59Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,0JDwY9o2QYJg11icyreRDd,False,You too (Feat. Cha Cha Malone),0,None,True,10,track,spotify:track:0JDwY9o2QYJg11icyreRDd,None
98,2017-05-26T10:18:15Z,False,None,https://open.spotify.com/user/mtmn_,https://api.spotify.com/v1/users/mtmn_,mtmn_,user,spotify:user:mtmn_,album,[{'external_urls': {'spotify': 'https://open.s...,...,30oZ161npcglwox5PhHiys,False,Rewind (Feat. SUMIN),0,None,True,11,track,spotify:track:30oZ161npcglwox5PhHiys,None


In [39]:
 df['added_by.id'].unique()

array(['mtmn_'], dtype=object)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   added_at                            100 non-null    object 
 1   is_local                            100 non-null    bool   
 2   primary_color                       0 non-null      object 
 3   added_by.external_urls.spotify      100 non-null    object 
 4   added_by.href                       100 non-null    object 
 5   added_by.id                         100 non-null    object 
 6   added_by.type                       100 non-null    object 
 7   added_by.uri                        100 non-null    object 
 8   track.album.album_type              99 non-null     object 
 9   track.album.artists                 100 non-null    object 
 10  track.album.available_markets       100 non-null    object 
 11  track.album.external_urls.spotify   99 non-nul

Colnames:

added_at


Left off working making function to get list of songs and features.

Currently need to find features needed to find songs. Look for important features available in the data.